In [31]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image

# Step 1: Data Preparation
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to a fixed size
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the images
])

train_dataset = datasets.ImageFolder(r'C:\Users\sskir\Desktop\3rdsem\workshop\data\train', transform=data_transforms)
val_dataset = datasets.ImageFolder(r'C:\Users\sskir\Desktop\3rdsem\workshop\data\valid', transform=data_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Step 2: Define the Network
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 112 * 112, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 16 * 112 * 112)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = MyNet()

# Step 3: Define the Loss Function
criterion = nn.CrossEntropyLoss()

# Step 4: Define the Optimization Algorithm
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Step 5: Training Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

# Step 6: Evaluate the Model
# You can test the model on a separate test set using a similar validation loop.

# Step 7: Save and Load the Model
torch.save(model.state_dict(), r'C:\Users\sskir\Desktop\3rdsem\workshop\model.pt')
# To load the model later:
# model = MyNet()
# model.load_state_dict(torch.load('path/to/save/model.pt'))

# Classify User-provided Images
model.eval()

# Load the saved model
model.load_state_dict(torch.load(r'C:\Users\sskir\Desktop\3rdsem\workshop\model.pt'))
model.to(device)

# Load and preprocess the user-provided image
user_image_path = r'C:\Users\sskir\Desktop\3rdsem\workshop\data\test\rama\rama (46).jpg'  # Replace with the path to the user's image
user_image = Image.open(user_image_path)
user_image = data_transforms(user_image).unsqueeze(0).to(device)

# Perform inference
with torch.no_grad():
    output = model(user_image)
    _, predicted = torch.max(output, 1)

# Get the predicted class label
predicted_label = train_dataset.classes[predicted.item()]
print("Predicted Label:", predicted_label)

Epoch [1/10], Train Loss: 0.6575, Train Accuracy: 0.5250, Val Loss: 0.3224, Val Accuracy: 1.0000
Epoch [2/10], Train Loss: 0.3054, Train Accuracy: 0.9125, Val Loss: 0.1285, Val Accuracy: 0.9000
Epoch [3/10], Train Loss: 0.1894, Train Accuracy: 0.9625, Val Loss: 0.0575, Val Accuracy: 1.0000
Epoch [4/10], Train Loss: 0.1658, Train Accuracy: 0.8875, Val Loss: 0.0299, Val Accuracy: 1.0000
Epoch [5/10], Train Loss: 0.1631, Train Accuracy: 0.9125, Val Loss: 0.0536, Val Accuracy: 1.0000
Epoch [6/10], Train Loss: 0.0555, Train Accuracy: 0.9875, Val Loss: 0.0241, Val Accuracy: 1.0000
Epoch [7/10], Train Loss: 0.0477, Train Accuracy: 1.0000, Val Loss: 0.0459, Val Accuracy: 1.0000
Epoch [8/10], Train Loss: 0.0152, Train Accuracy: 1.0000, Val Loss: 0.0085, Val Accuracy: 1.0000
Epoch [9/10], Train Loss: 0.0076, Train Accuracy: 1.0000, Val Loss: 0.0050, Val Accuracy: 1.0000
Epoch [10/10], Train Loss: 0.0048, Train Accuracy: 1.0000, Val Loss: 0.0061, Val Accuracy: 1.0000
Predicted Label: rama
